# Import the relevant Libraries

In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as ss
from bs4 import BeautifulSoup
import ast
from urllib.request import Request,urlopen
from itertools import combinations 

# Get header from .name file and add to dataset

In [ ]:
dataset = pd.read_csv('house-votes-84.data',header=None)
soup_link2 = BeautifulSoup(open('house-votes-84.names'))
table_header = soup_link2.find('p')

def before(value, a):
    pos_a = value.find(a)
    if pos_a == -1: return ""
    return value[0:pos_a]

def after(value, a):
    pos_a = value.rfind(a)
    if pos_a == -1: return ""
    adjusted_pos_a = pos_a + len(a)
    if adjusted_pos_a >= len(value): return ""
    return value[adjusted_pos_a:]

header = after(table_header.text,"7. Attribute Information:\n")
header = before(header,"\n8. Missing Attribute Values: Denoted by")
list_header = []
for i in range(header.count('\n')):
    if(i<10):
        buffer = after(header," "+str(i+1)+". ")
    else:
        buffer = after(header,str(i+1)+". ")
    buffer = before(buffer,":")
    list_header.append(str(buffer))

dataset.columns = list_header
dataset.head()

In [ ]:
dataset.shape

##### There are 434 rows and 17 columns 

In [ ]:
dataset.describe(include = 'object')

#### From the table above, we can see the basic data structure of dataset: 
#### All columns is categorical data
#### First column "republican" have 2 unique value which is (republican and democrat)
#### 16 other columns have 3 unique value which are y, n and ? (while ? is the missing value)

# Checking for missing values

#### Missing values is fill with "?" in the dataset so we have to take care of it


In [ ]:
## replace "?" value to most frequent values in each columns
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values = "?" , strategy = 'most_frequent',verbose=0)
imputer = imputer.fit(dataset.iloc[:,1:])
dataset.iloc[:, 1:] = imputer.transform(dataset.iloc[:, 1:])
display(dataset)

#### So that all the missing value is fill with the most frequent values
#### Because there only 2 unique values for all columns so we don't need dummy variable

# Calculate Correlation Matrix of dataset to check correlations among columns 

#### Because all columns of the dataset have categorical values so that we’re can not simply use corr() function of Pandas dataframe. we're looking for other measure of association between two categorical features.
#### By using Cramér’s V correlation which based on a nominal variation of Pearson’s Chi-Square Test will help us to handle this scenario

In [ ]:
def cramers_v(confusion_matrix):
    chi2 = ss.chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    phi2 = chi2/n
    r,k = confusion_matrix.shape
    phi2corr = max(0, phi2 - ((k-1)*(r-1))/(n-1))    
    rcorr = r - ((r-1)**2)/(n-1)
    kcorr = k - ((k-1)**2)/(n-1)
    return np.sqrt(phi2corr / min( (kcorr-1), (rcorr-1)))

cols = list(dataset.columns)
corrM = np.zeros((len(cols),len(cols)))
for col1, col2 in combinations(cols, 2):
    idx1, idx2 = cols.index(col1), cols.index(col2)
    corrM[idx1, idx2] = cramers_v(pd.crosstab(dataset[col1], dataset[col2]))
    corrM[idx2, idx1] = corrM[idx1, idx2]

corr = pd.DataFrame(corrM, index=cols, columns=cols)
display(corr.abs())


# Visualize Correlation Matrix of dataset by heat map

In [ ]:
sns.set(font_scale=2)
fig, ax = plt.subplots(figsize=(30, 30))
ax = sns.heatmap(corr, annot=True, ax=ax); ax.set_title("Cramer V Correlation between Variables");

# Selecting features based on correlation

#### We compare the correlation between features and remove one of two features that have a correlation higher than 0.9

In [ ]:
columns = np.full((corr.shape[0],), True, dtype=bool)
for i in range(corr.shape[0]):
    for j in range(i+1, corr.shape[0]):
        if corr.iloc[i,j] >= 0.9:
            if columns[j]:
                columns[j] = False
                selected_columns = dataset.columns[columns]
                new_dataset = dataset[selected_columns]
remove_columns = (list(set(dataset.columns) - set(new_dataset.columns)))
print(str(len(remove_columns))+" column removed which is: "+str(remove_columns))

#### Now dataset has only those columns with correlation less than 0.9

In [ ]:
display(new_dataset.head())

# Calculate distribution of selected features

#### We create new distribution dataframe for calculating

In [ ]:
buffer = pd.crosstab(index = new_dataset["Class Name"],columns=new_dataset["handicapped-infants"])
buffer = buffer.stack()
buffer.index = ['_'.join(idx) for idx in buffer.index]
buffer.name = "handicapped-infants"
distribution = pd.DataFrame(buffer)
for col in new_dataset.columns:
    if (col == "Class Name") or (col == "handicapped-infants"):
        pass
    else:
        buffer = pd.crosstab(index = new_dataset["Class Name"],columns=new_dataset[col])
        buffer = buffer.stack()
        buffer.index = ['_'.join(idx) for idx in buffer.index]
        distribution[col] = buffer.values 
display(distribution)

# Plotting the data to visualize their distribution

#### Using stacked bar to plot distribution

In [ ]:
sns.set(font_scale=1.5)
distribution.T.plot(kind='bar', stacked=True,figsize=(20,8),label='big')

In [ ]:
#### 1 ################################################################################################################
#### END 1

# Make classifier with SVM
#### Import libraries

In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix

#### Encode and split the training set

In [ ]:
le = LabelEncoder()
oe = OneHotEncoder()
training_dataset = new_dataset.copy()
dataset_features = training_dataset.drop(columns='Class Name')
dataset_classname = training_dataset['Class Name']
X = oe.fit_transform(dataset_features)
Y = le.fit_transform(dataset_classname)

#### Tune the model parameters

In [ ]:
param_grid = {'C': np.arange(1, 10, 0.3), 'gamma': ['scale', 'auto'], 'kernel': ['rbf', 'poly', 'sigmoid']}
grid = GridSearchCV(SVC(), param_grid, refit=True, verbose=0, cv=10, n_jobs=-1)
grid.fit(X, Y)
print("Best 10-fold accuracy:", grid.best_score_)
print(grid.best_estimator_)

In [ ]:
#### 7
#### END 7

In [ ]:
#### 8
#### END 8

In [ ]:
#### 9
#### END 9

In [ ]:
#### 10
#### END 10

In [ ]:
#### 11
#### END 11

In [ ]:
#### 12
#### END 12

In [ ]:
#### 13
#### END 13

In [ ]:
#### 14
#### END 14

In [ ]:
#### 15
#### END 15

In [ ]:
#### 16 ##############################################################################################################
#### END 16

In [ ]:
#### 17
#### END 17

# Make classifier with Naive Bayes

#### Encode Data

In [ ]:
from sklearn.preprocessing import LabelEncoder
import copy

def getEncodeData(DataFrame, Columns):
    labelEncoder = LabelEncoder()
    encodedData = copy.deepcopy(DataFrame)
    for column in Columns:
        encodedData[column] = labelEncoder.fit_transform(DataFrame[column])
    return encodedData

encodedDataSet = getEncodeData(dataset, list_header[1:])

x = encodedDataSet[list_header[1:]]  #Seperate data into input attributes and target attribute
y = encodedDataSet['Class Name'] 

#### Create classifier

In [ ]:
from sklearn.naive_bayes import GaussianNB

gaussianNaiveBayesModel = GaussianNB().fit(x, y.ravel())

#### Using 10-fold Cross Validation to test Accuracy

In [ ]:
from sklearn.model_selection import cross_val_score

gau_score = cross_val_score(gaussianNaiveBayesModel, x, y, cv =10)
print("Average score with Naive Bayes model: ", gau_score)

Average score with Naive Bayes model:  [0.95454545 0.93181818 0.97727273 0.81818182 0.97727273 0.90909091
 0.97727273 0.97674419 0.76190476 0.95238095]


#### Precision and Recall

In [30]:
import numpy as np
from sklearn import svm
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import precision_score, recall_score

skf = KFold(n_splits=10)

def precision_recall(model):
    precision_scores = []
    recall_scores = []
    for train_index, test_index in skf.split(x, y):    
        X_train, X_test = x.iloc[train_index], x.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        y_pred = model.predict(X_test)
        precision_scores.append(precision_score(y_test, y_pred, average='weighted'))
        recall_scores.append(recall_score(y_test, y_pred, average='weighted')) 
    print("Precision: ")
    print(precision_scores)
    print("Recall: ")
    print(recall_scores)

In [ ]:
precision_recall(gaussianNaiveBayesModel)     #Precicion and Recall for Naive Bayes Classfier

Precision: 
[0.9786096256684492, 0.9329945416901939, 0.9780058651026393, 0.8683418031244119, 0.9545454545454546, 0.9570661896243291, 0.9534883720930233, 1.0, 0.8567239635995956, 0.9309465524275804]
Recall: 
[0.9772727272727273, 0.9318181818181818, 0.9772727272727273, 0.8636363636363636, 0.9545454545454546, 0.9534883720930233, 0.9534883720930233, 1.0, 0.813953488372093, 0.9302325581395349]


In [60]:
#### 21
#### END 21

In [ ]:
#### 22
#### END 22

In [ ]:
#### 23
#### END 23

In [ ]:
#### 24
#### END 24

In [ ]:
#### 25
#### END 25

In [ ]:
#### 26
#### END 26

In [ ]:
#### 26
#### END 26

In [ ]:
#### 27
#### END 27

In [ ]:
#### 28
#### END 28

In [ ]:
#### 29
#### END 29

In [ ]:
#### 30
#### END 30

In [ ]:
#### 31 ###############################################################################################################
#### END 31

In [ ]:
#### 32
#### END 32

In [ ]:
#### 33
#### END 33

In [ ]:
#### 34
#### END 34

In [ ]:
#### 35
#### END 35

In [ ]:
#### 36
#### END 36

In [ ]:
#### 37
#### END 37

In [ ]:
#### 38
#### END 38

In [ ]:
#### 39
#### END 39

In [ ]:
#### 40
#### END 40

In [ ]:
#### 41
#### END 41

In [ ]:
#### 42
#### END 42

In [ ]:
#### 43
#### END 43

In [ ]:
#### 44
#### END 44

In [ ]:
#### 45
#### END 45